In [6]:
import os
import pandas as pd
from openpyxl import load_workbook
from clickhouse_driver import Client
import logging
from datetime import datetime

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
clickhouse_password = 'quie1ahpoo5Su0wohpaedae8keeph6bi'
database_name = 'default'
table_name = 'Vygruzka'

# Определение типов для известных столбцов
specified_columns = {
    'll': 'Nullable(Float64)',
    'oh': 'Nullable(Float64)',
    'sne': 'Nullable(Float64)',
    'ppr': 'Nullable(Float64)',
    'shop_visit_counter': 'Nullable(Float64)',
    'mfg_date': 'Nullable(Date)',
    'oh_at_date': 'Nullable(Date)',
    'removal_date': 'Nullable(Date)',
    'repair_date': 'Nullable(Date)',
    'Effectivity': 'Nullable(String)',
    'treshold': 'Nullable(Float64)'
}

# Поиск Excel файла в папке "Выгрузка"
folder_path = './Выгрузка'  # Папка "Выгрузка" на текущем уровне
excel_file = None

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.xlsx') or file.endswith('.xls'):
            excel_file = os.path.join(root, file)
            break
    if excel_file:
        break

if not excel_file:
    logging.error("Не найден ни один Excel файл в папке 'Выгрузка'")
else:
    # Загрузка данных из Excel с использованием openpyxl
    logging.info(f"Загрузка данных из файла: {excel_file}")
    df = pd.read_excel(excel_file, engine='openpyxl')
    logging.info(f"Считаны столбцы: {list(df.columns)}")
    
    # Получение всех столбцов из Excel файла
    all_columns = df.columns.tolist()
    
    # Формирование списка столбцов с типами для ClickHouse
    columns = []
    for col in all_columns:
        if col in specified_columns:
            columns.append((col, specified_columns[col]))
        else:
            columns.append((col, 'Nullable(String)'))
    
    # Подключение к ClickHouse
    client = Client(host=clickhouse_host, port=clickhouse_port, user=clickhouse_user, password=clickhouse_password)

    # Удаление существующей таблицы (если необходимо)
    client.execute(f"DROP TABLE IF EXISTS {database_name}.{table_name}")

    # Функция для создания таблицы с динамическими столбцами
    def create_table(client, columns):
        fields = ', '.join([f"{name} {dtype}" for name, dtype in columns])
        query = f"""
        CREATE TABLE IF NOT EXISTS {database_name}.{table_name} (
            {fields}
        ) ENGINE = MergeTree()
        ORDER BY tuple()
        """
        client.execute(query)
    
    # Создание таблицы
    create_table(client, columns)

    # Преобразование данных
    data = df.copy()
    
    # Обработка столбцов дат
    date_columns = ['mfg_date', 'oh_at_date', 'removal_date', 'repair_date']
    for col in date_columns:
        if col in data.columns:
            data[col] = pd.to_datetime(data[col], dayfirst=True, errors='coerce').dt.date
            data[col] = data[col].where(data[col].notnull(), None)
    
    # Обработка числовых столбцов
    float_columns = ['ll', 'oh', 'sne', 'ppr', 'shop_visit_counter', 'treshold']
    for col in float_columns:
        if col in data.columns:
            data[col] = pd.to_numeric(data[col], errors='coerce')
            data[col] = data[col].where(data[col].notnull(), None)
    
    # Обработка строковых столбцов
    string_columns = [col for col in data.columns if col not in float_columns + date_columns]
    for col in string_columns:
        if col in data.columns:
            data[col] = data[col].astype(str).where(data[col].notnull(), None)
    
    # Заполнение пропущенных значений
    data.fillna(value=pd.NA, inplace=True)
    
    # Преобразование данных в список словарей
    records = data.to_dict('records')
    
    # Вставка данных в таблицу
    try:
        client.execute(f"INSERT INTO {database_name}.{table_name} VALUES", [tuple(record.values()) for record in records])
        logging.info("Данные успешно загружены в таблицу ClickHouse")
    except Exception as e:
        logging.error(f"Произошла ошибка при вставке данных: {e}", exc_info=True)


2024-12-02 04:25:01,556 - INFO - Загрузка данных из файла: ./Выгрузка/Standard Table Report 251124 ВНВ rev KDV.xlsx
2024-12-02 04:25:01,629 - INFO - Считаны столбцы: ['partno', 'serialno', 'ac_typ', 'location', 'll', 'oh', 'sne', 'ppr', 'mfg_date', 'oh_at_date', 'shop_visit_counter', 'owner', 'condition', 'removal_date', 'repair_date', 'Effectivity', 'threshold']
2024-12-02 04:25:01,676 - INFO - Данные успешно загружены в таблицу ClickHouse
